In [ ]:
import polars as pl
import pandas as pd # Para el guardado final del CSV si se prefiere
import os

In [ ]:
# --- Rutas (Asegúrate que existan) ---
RUTA_DATOS_INPUT = './datos/' # Donde están DIM_TIENDA_TEST.csv, venta.csv, Meta_venta.csv
RUTA_DATOS_OUTPUT_PROCESADO = './datos/output/procesado_para_modelado/' # Donde se guardará y_test_oficial_real.csv

# Crear directorio de output si no existe
if not os.path.exists(RUTA_DATOS_OUTPUT_PROCESADO):
    os.makedirs(RUTA_DATOS_OUTPUT_PROCESADO)
    print(f"Directorio creado: {RUTA_DATOS_OUTPUT_PROCESADO}")

# --- Nombres de Archivos ---
ARCHIVO_DIM_TIENDA_TEST = RUTA_DATOS_INPUT + 'DIM_TIENDA_TEST.csv'
ARCHIVO_VENTA = RUTA_DATOS_INPUT + 'venta.csv' # Asumimos que es el mismo archivo de ventas
ARCHIVO_META_VENTA = RUTA_DATOS_INPUT + 'Meta_venta.csv'

ARCHIVO_SALIDA_Y_TEST = RUTA_DATOS_OUTPUT_PROCESADO + 'y_test_oficial_real.csv'

# --- Columnas Clave (Verifica que coincidan con tus archivos) ---
# En DIM_TIENDA_TEST.csv
COL_TIENDA_ID_DIM = 'TIENDA_ID'
COL_ENTORNO_DIM = 'ENTORNO_DES'
# En venta.csv
COL_TIENDA_ID_VENTA = 'TIENDA_ID'
COL_MES_ID_VENTA = 'MES_ID'
COL_VENTA_TOTAL_VENTA = 'VENTA_TOTAL'
# En Meta_venta.csv
COL_ENTORNO_META = 'ENTORNO_DES'
COL_META_VALOR_META = 'Meta_venta'

# --- Definición del Periodo de Ventas (DEBE SER IDÉNTICO AL USADO PARA EL TRAIN SET) ---
# Ejemplo: Agosto 2023 a Julio 2024 (12 meses)
# O Enero 2023 a Julio 2024 (19 meses) si fue lo que usaste
MES_INICIO_PERIODO = 202301 # Enero 2023 (para 19 meses hasta Julio 2024)
MES_FIN_PERIODO = 202412   # Julio 2024

print(f"--- Creando Target para DIM_TIENDA_TEST.csv ---")
print(f"Usando periodo de ventas: de {MES_INICIO_PERIODO} a {MES_FIN_PERIODO}")

In [ ]:
# --- 1. Cargar los archivos ---
try:
    # Usar Polars para todos por consistencia y eficiencia con venta.csv
    dim_tienda_test_lf = pl.scan_csv(ARCHIVO_DIM_TIENDA_TEST)
    lf_venta = pl.scan_csv(ARCHIVO_VENTA) 
    meta_venta_lf = pl.scan_csv(ARCHIVO_META_VENTA)
    print("Archivos CSV escaneados exitosamente con Polars.")
except Exception as e:
    print(f"Error al escanear archivos: {e}")
    raise

# Verificar columnas en lf_venta
schema_venta = lf_venta.collect_schema()
if COL_VENTA_TOTAL_VENTA not in schema_venta.names() or \
   COL_MES_ID_VENTA not in schema_venta.names() or \
   COL_TIENDA_ID_VENTA not in schema_venta.names():
    print(f"Error: Columnas esperadas no encontradas en '{ARCHIVO_VENTA}'.")
    print(f"Detectadas: {schema_venta.names()}. Esperadas: '{COL_TIENDA_ID_VENTA}', '{COL_MES_ID_VENTA}', '{COL_VENTA_TOTAL_VENTA}'")
    raise ValueError(f"Columnas faltantes en '{ARCHIVO_VENTA}'")

In [ ]:
# --- 2. Procesar lf_venta para obtener ventas agregadas para las tiendas de TEST ---
# Asegurar que MES_ID sea del tipo correcto para la comparación
lf_venta_periodo = (
    lf_venta
    .with_columns(pl.col(COL_MES_ID_VENTA).cast(pl.Int64, strict=False)) # Cast a Int64, si falla no es crítico aquí pero sí para el filtro
    .filter(
        (pl.col(COL_MES_ID_VENTA) >= MES_INICIO_PERIODO) & (pl.col(COL_MES_ID_VENTA) <= MES_FIN_PERIODO)
    )
)

ventas_agregadas_test_lf = (
    lf_venta_periodo
    .group_by(COL_TIENDA_ID_VENTA)
    .agg(
        pl.col(COL_VENTA_TOTAL_VENTA).mean().alias("VENTA_PROMEDIO_MENSUAL"),
        pl.count().alias("N_MESES_CON_VENTA_EN_PERIODO_TEST") 
    )
)

# Ejecutar el cálculo y obtener un DataFrame de Polars
ventas_agregadas_test_df = ventas_agregadas_test_lf.collect()
print("\nVentas promedio mensual calculadas para tiendas en el archivo de ventas:")
print(ventas_agregadas_test_df.head())
print(f"Total de tiendas con ventas en el periodo: {len(ventas_agregadas_test_df)}")

In [ ]:
# --- 3. Unir (join) los DataFrames ---
# Primero, obtener el DataFrame de tiendas de test
dim_tienda_test_df_collected = dim_tienda_test_lf.select([COL_TIENDA_ID_DIM, COL_ENTORNO_DIM]).collect()
# Renombrar columnas si es necesario para el join
dim_tienda_test_df_collected = dim_tienda_test_df_collected.rename({
    COL_TIENDA_ID_DIM: "TIENDA_ID", # Estandarizar a "TIENDA_ID"
    COL_ENTORNO_DIM: "ENTORNO_DES"  # Estandarizar a "ENTORNO_DES"
})


# Unir tiendas de test con sus ventas promedio
# Solo nos interesan las tiendas que están en DIM_TIENDA_TEST.csv
tiendas_test_con_ventas_df = dim_tienda_test_df_collected.join(
    ventas_agregadas_test_df, 
    on="TIENDA_ID", # Usar el nombre estandarizado
    how="left" # Mantener todas las tiendas de test, algunas podrían no tener ventas
)
print(f"\nTiendas TEST después de unir con ventas: {len(tiendas_test_con_ventas_df)}")

# Cargar y unir con las metas de venta
meta_venta_df_collected = meta_venta_lf.select([COL_ENTORNO_META, COL_META_VALOR_META]).collect()
meta_venta_df_collected = meta_venta_df_collected.rename({
    COL_ENTORNO_META: "ENTORNO_DES",      # Estandarizar
    COL_META_VALOR_META: "Meta_venta" # Estandarizar
})


tiendas_test_con_metas_df = tiendas_test_con_ventas_df.join(
    meta_venta_df_collected, 
    on="ENTORNO_DES", 
    how="left"
)
print(f"Tiendas TEST después de unir con metas: {len(tiendas_test_con_metas_df)}")

# Manejar NaNs que pudieron surgir de los joins (tiendas sin ventas o sin meta por entorno)
print(f"\nNaNs en VENTA_PROMEDIO_MENSUAL (TEST): {tiendas_test_con_metas_df['VENTA_PROMEDIO_MENSUAL'].is_null().sum()}")
print(f"NaNs en Meta_venta (TEST): {tiendas_test_con_metas_df['Meta_venta'].is_null().sum()}")

tiendas_test_completas_df = tiendas_test_con_metas_df.drop_nulls(subset=['VENTA_PROMEDIO_MENSUAL', 'Meta_venta'])
print(f"Tiendas TEST después de eliminar NaNs en ventas/metas: {len(tiendas_test_completas_df)}")

In [ ]:
# --- 4. Crear la variable EXITO_REAL ---
if tiendas_test_completas_df.height > 0:
    tiendas_test_completas_df = tiendas_test_completas_df.with_columns(
        EXITO_REAL = (pl.col('VENTA_PROMEDIO_MENSUAL') >= pl.col('Meta_venta')).cast(pl.Int8)
    )
    print("\nVariable EXITO_REAL creada para el conjunto de test.")
    print(tiendas_test_completas_df.select(['TIENDA_ID', 'VENTA_PROMEDIO_MENSUAL', 'Meta_venta', 'N_MESES_CON_VENTA_EN_PERIODO_TEST', 'EXITO_REAL']).head())
    print("\nDistribución de la variable EXITO_REAL:")
    print(tiendas_test_completas_df['EXITO_REAL'].value_counts(sort=True)) # Polars usa sort=True para value_counts
else:
    print("DataFrame 'tiendas_test_completas_df' está vacío. No se pudo crear la variable EXITO_REAL.")
    raise ValueError("No se pudieron generar datos completos para crear EXITO_REAL para el test set.")


In [ ]:
# --- 5. Seleccionar columnas TIENDA_ID y EXITO_REAL y guardar ---
# Asegurarse de que TIENDA_ID y EXITO_REAL existan
if "TIENDA_ID" not in tiendas_test_completas_df.columns or "EXITO_REAL" not in tiendas_test_completas_df.columns:
    raise ValueError("Columnas TIENDA_ID o EXITO_REAL no encontradas en el DataFrame final.")

y_test_oficial_real_df_pl = tiendas_test_completas_df.select(["TIENDA_ID", "EXITO_REAL"])

# Convertir a Pandas para guardar como CSV (opcional, Polars también puede guardar CSV)
y_test_oficial_real_df_pd = y_test_oficial_real_df_pl.to_pandas()

try:
    y_test_oficial_real_df_pd.to_csv(ARCHIVO_SALIDA_Y_TEST, index=False)
    print(f"\nArchivo '{ARCHIVO_SALIDA_Y_TEST}' guardado exitosamente.")
    print(f"Número de tiendas en el archivo de target de test: {len(y_test_oficial_real_df_pd)}")
except Exception as e:
    print(f"Error al guardar el archivo: {e}")

print(f"\n--- Creación de Target para DIM_TIENDA_TEST.csv Completada ---")